![](https://miro.medium.com/max/1200/1*r2eKvfvYPQuizKLOh9q7Hw.jpeg)

# What is PyTorch ?

* PyTorch is an open source machine learning library based on the Torch library, used for applications such as computer vision and natural language processing, primarily developed by Facebook's AI Research lab

# Advantages of Pytorch.

*  PyTorch is Pythonic
* Easy to learn
* Higher developer productivity
* Easy debugging
* Data Parallelism
* Dynamic Computational Graph Support
* Fast model training 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')

In [ ]:
data.head()

### Checking for Null values.

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

### Checking the shape of the dataset

In [ ]:
data.shape

### Plotting a Correlation Matrix

In [ ]:
corr_mat = data.corr()
top_corr_features = corr_mat.index

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(data[top_corr_features].corr(), annot=True, cmap='RdYlGn')

Plotting the target variable

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='target', data=data, palette='RdBu_r')

In [ ]:
data.head()


### As age, trestbps, chol, thalach, oldpeak has higher values we need to scale down the values.

### Before feeding our data to our Neural Network are values should be scaled 

In [ ]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
columns_to_scale = ['age','trestbps','chol','thalach','oldpeak']
data[columns_to_scale] = standard_scaler.fit_transform(data[columns_to_scale])

In [ ]:
data.head()

### One Hot Encoding the categorical Columns

In [ ]:
data = pd.get_dummies(data, columns=['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope','ca', 'thal'])

In [ ]:
data.head()

### Splitting the data into training and validation set.

In [ ]:
X = data.drop(['target'], axis=1).values
y = data['target'].values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

### Importing out torch deep learning library

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Converting our data into tensors(very inportant)

* All your inputs and outputs ie. your independent and dependent features should be converted into tensors.
* Your independent features needs to be converted into float tensor its very important
* With respect to your dependent features its not that important

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train) ## LongTensor dosent converts it into floating point 
y_test = torch.LongTensor(y_test)

### Creating our Neural Network using Torch

In [ ]:
class Net(nn.Module):
    def __init__(self, in_count, output_count):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_count, 50)
        self.fc2 = nn.Linear(50, 25)
        self.fc3 = nn.Linear(25, output_count)
        self.softmax = nn.Softmax()
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return self.softmax(x)

### Passing the data to our Neural Network

In [ ]:
model = Net(X.shape[1], 2) 

### Defining the criterion and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Training our Neural Network

In [ ]:
final_losses = []
for epochs in range(5000):
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out, y_train)
    final_losses.append(loss)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epochs+1}, loss: {loss.item()}")


### Plotting our losses

In [ ]:
plt.plot(range(5000), final_losses)
plt.ylabel('loss')

* We can see from the graph is that the losses have decreased significantly.

### Evaluating the accuracy

In [ ]:
from sklearn.metrics import accuracy_score

pred = model(X_test)
_, predict_classes = torch.max(pred,1)

correct = accuracy_score(y_test, predict_classes)
print(f"Accurcay :   {correct}")

### Making predictions on the x_test

In [ ]:
predictions = []

with torch.no_grad():
    for i, data in enumerate(X_test):
        y_pred = model(data)
        predictions.append(y_pred.argmax().item())
        print(y_pred.argmax().item())

### Saving the model

In [ ]:
torch.save(model, 'heart-disease.pt')


### Plotting the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
cm

In [ ]:
sns.heatmap(cm, annot=True)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')

### Checking Precision

In [ ]:
from sklearn.metrics import precision_score

pre = precision_score(y_test, predictions)
pre

### Checking for Recall

In [ ]:
from sklearn.metrics import recall_score

rc = recall_score(y_test, predictions)
rc

# If you like it please consider upvoting it Thank You :)
# Code link

* GitHUb : https://github.com/akhiilkasare/Heart-Disease-Prediction-Using-Pytorch